# Object Oriented Deep Neural Networks

<img src="https://images.squarespace-cdn.com/content/v1/565272dee4b02fdfadbb3d38/1521403348811-P4RMPTMIIVYZ84AHJKKL/ke17ZwdGBToddI8pDm48kMdFogk4zjxAyRDEXPTJab0UqsxRUqqbr1mOJYKfIPR7LoDQ9mXPOjoJoqy81S2I8N_N4V1vUb5AoIIIbLZhVYxCRW4BPu10St3TBAUQYVKcSBJjw6RvutB-lteUIUBBCmTHGjj7_rH39SC6UGXex2iIDOf1Tmg0PX5LsnDS6ZM3/Blank+Diagram+-+Page+1.png?format=750w" width="480" border="1"/>


The effort of building, expanding and maintaining complex deep neural network architectures can be alleviated using principles of **Object Oriented Programming (OOP)**. This approach belongs to the domain of **Machine Learning Engineering** combines the best of at least two worlds: Software Engineering + Machine Learning.

*(Objective: Illustrate reusability and the implementation of a complex model with readable code. Time: 10 mins)*

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/OOT2019/lib')

# Display the system path elements
for sFolder in sys.path:
  print(sFolder)

Install ngrok

In [0]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -o ngrok-stable-linux-amd64.zip

## Object oriented model declaration

When declaring the layers of a neural network model there is a lot of repetition which suits to the **DRY (Don't Repeat Yourself) principle** of Object Oriented Programming. We can **encapsulate** the whole declaration inside an object for the model, cleaning up the main method of our program. Using **inheritance** we can make the basics of neural networks reusable and facilitate the declaration of deeper and more complex networks.

### Implementation Notes:
*   The logic for declaring the tensors of a fully connected layer in placed in the ancestor class `:NeuralNetwork`. 
*   The descendand class `:DeeperNetwork ` overrides a virtual method ` CreateModel()` to create its custom architecture.
*   The weights of the fully connected layer are initialized randomly from a normal distribution with specified mean and standard deviation. Any values whose magnitude is more than 2 standard deviations from the mean are dropped and re-picked (truncated).

### Theory: The choice for activation function
*   If we use sigmoidal activation functions in deeper networks we face the [Vanishing Gradients Problem](https://towardsdatascience.com/the-vanishing-gradient-problem-69bf08b15484). This is why rectifiers are preferred.

     <img src="https://qph.fs.quoracdn.net/main-qimg-07bc0ec05532caf5ebe8b4c82d0f5ca3" width="500" border="1"/>

In [0]:
# NEURAL NETWORK - Object Oriented

import tensorflow.compat.v1 as tf
import math

TENSORBOARD_FOLDER = "/tmp/dnn"

#==================================================================================================
class NeuralNetwork(object):
    #------------------------------------------------------------------------------------
    def __init__(self):
        #........ |  Instance Attributes | ..............................................
        # // Collections that keep the neural network model parameters \\
        self.FCWeights = []
        self.FCBiases  = []
        #................................................................................
        
        # This is a call to virtual method that descendants override
        self.CreateModel()
    #------------------------------------------------------------------------------------
    def CreateModel(self):
        pass
    #------------------------------------------------------------------------------------
    def GetParameter(self, p_tShape, p_tInitializer=tf.initializers.constant(0.0), p_bIsBias=False):
      if p_bIsBias:
        sParamName = "b"
      else:
        sParamName = "w"
    
      tParam = tf.get_variable(sParamName, shape=p_tShape, initializer=p_tInitializer)
      return tParam
    #------------------------------------------------------------------------------------
    def FullyConnected(self, p_tInput, p_nNeuronCount, p_tActivationFunction=None):
      nLayerNum = len(self.FCWeights) + 1
      nInputNeurons = p_tInput.get_shape().as_list()[-1]
      
      with tf.variable_scope("FC%d" % nLayerNum):
        tW = self.GetParameter([nInputNeurons, p_nNeuronCount], tf.initializers.truncated_normal(mean=0.0, stddev=math.sqrt(2/(nInputNeurons+p_nNeuronCount)))) 
        tB = self.GetParameter([p_nNeuronCount], p_bIsBias=True ) 
        tU = tf.matmul(p_tInput, tW) + tB
        
        if p_tActivationFunction is not None:
          # Activation function is a function reference that is passed to the method
          tA = p_tActivationFunction(tU)
        else:
          tA = tU
      
        self.FCWeights.append(tW)
        self.FCBiases.append(tB)
      
      return tA
    #------------------------------------------------------------------------------------
    
#==================================================================================================




    
#==================================================================================================  
class DeeperNetwork(NeuralNetwork):  
    #------------------------------------------------------------------------------------
    def __init__(self, p_nFeatures=[128,256,512,10]):
        #........ |  Instance Attributes | ..............................................
        # // Network input and output tensors \\
        self.Input      = None
        self.Logits     = None
        self.Prediction = None
        
        # // Architectural hyperparameters \\
        self.Features = p_nFeatures
        #................................................................................
        
        # Invoke the inherited logic from ancestor :NeuralNetwork
        super(DeeperNetwork, self).__init__()
    #------------------------------------------------------------------------------------
    def CreateModel(self):
      with tf.variable_scope("NeuralNet"):
        with tf.variable_scope("Input"):
          self.Input = tf.placeholder(tf.uint8, shape=(100,768,3))
          tInputNormalized = tf.cast(tf.cast(self.Input, tf.float32) / 255.0, tf.float32)
          tX = tInputNormalized
        
        nLayerIndex = 1
        with tf.variable_scope("L%d" % nLayerIndex):
          tA = tf.nn.relu(self.FullyConnected(tX, self.Features[0]))
        
        # Using same feature depth inside the context of a module (layers 2-9 and 10-17)
        with tf.variable_scope("Module1"):
            for nLayerIndex in range(2,10):
              with tf.variable_scope("L%d" % nLayerIndex):
                tA = tf.nn.relu(self.FullyConnected(tA, self.Features[1]))
                          
        with tf.variable_scope("Module2"):
            for nLayerIndex in range(10, 18):
              with tf.variable_scope("L%d" % nLayerIndex):              
                tA = tf.nn.relu(self.FullyConnected(tA, self.Features[2]))
        
        with tf.variable_scope("Classifier"):
          self.Logits     = self.FullyConnected(tA, self.Features[-1])
          self.Prediction = tf.nn.softmax(self.Logits)
    #------------------------------------------------------------------------------------
    
#==================================================================================================  

  
  
  


#------------------------------------------------------------------------------------
def Main():
  oGraph = tf.Graph()
  with oGraph.as_default():
    oNet = DeeperNetwork([128,256,512,10])

    with tf.Session() as oSession:
      assert oSession.graph == oGraph, "The current session is using the default graph"
      oSession.run(tf.initializers.global_variables())


      oWriter = tf.summary.FileWriter(TENSORBOARD_FOLDER, graph=oSession.graph, flush_secs=20)
      oWriter.flush()
      print("Graph exported to %s" % TENSORBOARD_FOLDER)
#------------------------------------------------------------------------------------



    
if __name__ == '__main__':
  Main()

Display the graph in Tensorboard

In [0]:
# kill all running ngrok instances
!pkill -f ngrok
!pkill -f tensorboard

# Execute tensorboard
LOG_DIR = '/tmp/dnn/'
get_ipython().system_raw('tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'.format(TENSORBOARD_FOLDER))

# execute ngrok
get_ipython().system_raw('./ngrok http 6006 &')

# Do the tunneling
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

We move the ancestor class for neural networks `:NeuralNetwork ` in the **oot.nn** namespace to make it reusable. Our source code becomes even more simplified, keeping only the custom implementation. We improve the graph by adding extra hierarchy in the variable scopes

In [0]:
# NEURAL NETWORK - Object Oriented

import tensorflow.compat.v1 as tf
import math
from ootf.nn import NeuralNetwork


TENSORBOARD_FOLDER = "/tmp/dnn"

    
#==================================================================================================  
class DeeperNetwork(NeuralNetwork):  
    #------------------------------------------------------------------------------------
    def __init__(self, p_nFeatures=[128,256,512,10]):
        #........ |  Instance Attributes | ..............................................
        # // Network input and output tensors \\
        self.Input      = None
        self.Logits     = None
        self.Prediction = None
        
        # // Architectural hyperparameters \\
        self.Features = p_nFeatures
        #................................................................................
        
        # Invoke the inherited logic from ancestor :NeuralNetwork
        super(DeeperNetwork, self).__init__()
    #------------------------------------------------------------------------------------
    def CreateModel(self):
      with tf.variable_scope("NeuralNet"):
        with tf.variable_scope("Input"):
          self.Input = tf.placeholder(tf.uint8, shape=(100,768,3))
          tInputNormalized = tf.cast(tf.cast(self.Input, tf.float32) / 255.0, tf.float32)
          tX = tInputNormalized
        
        nLayerIndex = 1
        with tf.variable_scope("L%d" % nLayerIndex):
          tA = tf.nn.relu(self.FullyConnected(tX, self.Features[0]))
        
        # Using same feature depth inside the context of a module (layers 2-9 and 10-17)
        with tf.variable_scope("Module1"):
            for nLayerIndex in range(2,10):
              with tf.variable_scope("L%d" % nLayerIndex):
                tA = tf.nn.relu(self.FullyConnected(tA, self.Features[1]))
                          
        with tf.variable_scope("Module2"):
            for nLayerIndex in range(10, 18):
              with tf.variable_scope("L%d" % nLayerIndex):              
                tA = tf.nn.relu(self.FullyConnected(tA, self.Features[2]))
        
        with tf.variable_scope("Classifier"):
          self.Logits     = self.FullyConnected(tA, self.Features[-1])
          self.Prediction = tf.nn.softmax(self.Logits)
    #------------------------------------------------------------------------------------
    
#==================================================================================================  


#------------------------------------------------------------------------------------
def Main():
  oGraph = tf.Graph()
  with oGraph.as_default():
    oNet = DeeperNetwork([128,256,512,10])

    with tf.Session() as oSession:
      assert oSession.graph == oGraph, "The current session is using the default graph"
      oSession.run(tf.initializers.global_variables())


      oWriter = tf.summary.FileWriter(TENSORBOARD_FOLDER, graph=oSession.graph, flush_secs=20)
      oWriter.flush()
      print("Graph exported to %s" % TENSORBOARD_FOLDER)
#------------------------------------------------------------------------------------



    
if __name__ == '__main__':
  Main()